In [ ]:
from visualization_fct import *

from pomegranate import HiddenMarkovModel
from pomegranate import State, Distribution, MultivariateGaussianDistribution

from bokeh.plotting import output_notebook, show

%matplotlib notebook
output_notebook()

In [ ]:
data = pd.read_csv("./asm_data_for_ml.txt", sep='\t')
del data['MJD']
del data['error']
del data['errorA']
del data['errorB']
del data['errorC']

data['rateCA'] = data.rateC / data.rateA

data_thr = mask(data, 'orbit')  # rm too large values except for 'orbit'

In [ ]:
# Hmm with 3 components:
np.random.seed(2)

# looks better with data.rateCA in X
# looks like 3 is better than 4 components

from pomegranate import HiddenMarkovModel
from pomegranate import State, Distribution

#TODO kmeans

X = np.c_[data.orbit, data.rateA, data.rateB, data.rateC, data.rate, data.rateCA]

X_1 = X[:1000]
X_2 = X[1000:2000]
X_3 = X[2000:3000]
a = MultivariateGaussianDistribution.from_samples(X_1)   
b = MultivariateGaussianDistribution.from_samples(X_2)  
c = MultivariateGaussianDistribution.from_samples(X_3)
s1 = State(a, name="M1" )
s2 = State(b, name="M2" )
s3 = State(c, name="M3" )

hmm = HiddenMarkovModel()
hmm.add_states(s1, s2, s3)
hmm.add_transition( hmm.start, s1, 0.5 )
hmm.add_transition( hmm.start, s3, 0.5 )
hmm.add_transition( s1, s1, 0.8 )
hmm.add_transition( s1, s2, 0.2 )
hmm.add_transition( s2, s1, 0.5 )
hmm.add_transition( s2, s3, 0.5 )
hmm.add_transition( s3, s3, 0.8 )
hmm.add_transition( s3, s2, 0.2 )
hmm.bake()
hmm.fit(X)
preds = hmm.predict(X)
probs = hmm.predict_proba(X)

In [ ]:
data_thr['preds'] = pd.Series(preds).astype("category")
color_key = ['red', 'blue', 'yellow']
p = scatter_matrix(data_thr, spread=True, color_key=color_key)
show(p)

In [ ]:
fig = plot_probs_datashader(probs)
show(fig)

In [ ]:
q = scatter_matrix_seaborn(data_thr)

In [ ]:
plot_probas(data_thr, probs, color_key=color_key)